In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

### Defining/loading stuff

In [ ]:
data_features = np.load('BW_miniimagenet_3600_60_60.npy').reshape((3600,3600))

In [ ]:
def corrupt_p(codebook,p=0.1,booktype='01'):
  rand_indices = np.sign(np.random.uniform(size=codebook.shape)- p )
  if booktype=='-11':
    return np.multiply(codebook,rand_indices)
  elif booktype=='01':
    return abs(codebook - 0.5*(-rand_indices+1))
  elif booktype=='cts':
    return codebook + np.random.normal(0,1,size=codebook.shape)*p
  else:
    print("codebook should be -11; 01; or cts")
    return 0


In [ ]:
sbook = data_features - np.mean(data_features.flatten())

In [ ]:
sbook = np.sign(np.random.uniform(size=(Npatts,Ns))-0.5)
sbook = 0.5*(sbook+1)

In [ ]:
def cleanup(s, sbook):
  idx = np.argmax(sbook@s)
  sclean = sbook[idx,:]
  return sclean

In [ ]:
def binarize(data,bin_type='01'):
  if bin_type=='01':
    return 0.5*(np.sign(data - 0.5)+1)
  elif bin_type=='-11':
    return np.sign(data)

In [ ]:
def get_error(x,y):
  if x.shape != y.shape:
    return "error"
  return np.sum(abs(x-y))/np.prod(x.shape)

In [ ]:
def get_mse(x,y):
  if x.shape != y.shape:
    return "error"
  return np.average((x-y)**2.)

In [ ]:
def get_overlap(x,y):
  if x.shape != y.shape:
    return "error"
  x = (x/np.linalg.norm(x,axis=1)[:,None])
  y = (y/np.linalg.norm(y,axis=1)[:,None])
  return np.average(np.einsum('ij,ij->i',x,y))

### Varying Npatts binary sbook seq

In [ ]:
###Sequences

import tqdm
Npatts_list = np.arange(1,3601,50)
nruns = 3
first_clean_overlap_list = np.zeros((Npatts_list.shape[0],nruns))
final_clean_overlap_list = np.zeros((Npatts_list.shape[0],nruns))
first_noisy_overlap_list = np.zeros((Npatts_list.shape[0],nruns))
final_noisy_overlap_list = np.zeros((Npatts_list.shape[0],nruns))
verbose=False

for nidx in (range(nruns)):
  for idx,Npatts in enumerate(tqdm.tqdm(Npatts_list)):
    if verbose:
      print("xxxxx nidx = "+str(nidx))
      print("xxxxx idx = "+str(idx))
      print("xxxxx Npatts = "+str(Npatts))
    Ng=38#9+16+25
    Np=275#400
    Ns=900#3600
    # Npatts=200
    data_shape = Ns
    test_noise_frac=0.0
    test_noise_frac1=0.0#0.025   ###Second case will be used to apply sign nonlinearity at each timestep

    sbook = np.sign(np.random.uniform(size=(Npatts,Ns))-0.5)
    

    true_data = sbook[:Npatts]   #data_features[:Npatts]   #double_train_data_cts  #train_data_cts   #noise_aug_train_data  #sbook # bin_train[:Npatts] #

    class Autoencoder(Model):
      def __init__(self, Np,Ng,Ns):
        super(Autoencoder, self).__init__()
        self.Np = Np
        self.Ns = Ns
        self.Ng = Ng
        self.encoder = tf.keras.Sequential([
          layers.Dense(Np, activation='tanh'),
          layers.Dense(Ng, activation='tanh'),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(Np, activation='tanh'),
          layers.Dense(Ns, activation='tanh')
        ])

      def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    autoencoder = Autoencoder(Np,Ng,Ns)

    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

    autoencoder.fit(true_data[:-1], true_data[1:], epochs=1000, verbose=0);
    
    # true_data = sbook  #train_data  #
    if verbose: print("-------------Testing--1----------------")
    if verbose: print("Testing noise percentage = "+str(test_noise_frac))
    cts_corrupt = corrupt_p(true_data,p=test_noise_frac,booktype='-11')

    decoded_data = np.zeros_like(true_data)
    decoded_data[0] = cts_corrupt[0]

    
    for ite in range(1,Npatts):
      encoded_data = autoencoder.encoder(decoded_data[ite-1,None]).numpy()
      decoded_data[ite] = autoencoder.decoder(encoded_data).numpy()
    decoded_data = np.sign(decoded_data)

    if verbose:
      print("Np = "+str(Np))
      print("Ns = "+str(Ns))
      print("Ng = "+str(Ng))
      print("Npatts = "+str(Npatts))


      print("Initial error: " + str(get_mse(true_data[:,:data_shape], cts_corrupt[:Npatts,:data_shape])))
      print("error after one timestep: " + str(get_mse(true_data[:Npatts,:data_shape],decoded_data1[:Npatts,:data_shape])))
      print("error after iteration: " + str(get_mse(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])))

      print("Initial overlap: " + str(get_overlap(true_data[:,:data_shape], cts_corrupt[:Npatts,:data_shape])))
      print("overlap after one timestep: " + str(get_overlap(true_data[:Npatts,:data_shape],decoded_data1[:Npatts,:data_shape])))
      print("overlap after iteration: " + str(get_overlap(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])))

    final_clean_overlap_list[idx,nidx]=get_overlap(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])

    if verbose: print("-------------Testing--2----------------")
    if verbose: print("Testing noise percentage = "+str(test_noise_frac1))
    cts_corrupt = corrupt_p(true_data,p=test_noise_frac1,booktype='-11')

    decoded_data = np.zeros_like(true_data)
    decoded_data[0] = cts_corrupt[0]

    
    for ite in range(1,Npatts):
      encoded_data = autoencoder.encoder(decoded_data[ite-1,None]).numpy()
      decoded_data[ite] = np.sign(autoencoder.decoder(encoded_data).numpy())
    decoded_data = np.sign(decoded_data)

    if verbose:
      print("Np = "+str(Np))
      print("Ns = "+str(Ns))
      print("Ng = "+str(Ng))
      print("Npatts = "+str(Npatts))


      print("Initial error: " + str(get_mse(true_data[:,:data_shape], cts_corrupt[:Npatts,:data_shape])))
      print("error after one timestep: " + str(get_mse(true_data[:Npatts,:data_shape],decoded_data1[:Npatts,:data_shape])))
      print("error after iteration: " + str(get_mse(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])))

      print("Initial overlap: " + str(get_overlap(true_data[:,:data_shape], cts_corrupt[:Npatts,:data_shape])))
      print("overlap after one timestep: " + str(get_overlap(true_data[:Npatts,:data_shape],decoded_data1[:Npatts,:data_shape])))
      print("overlap after iteration: " + str(get_overlap(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])))

    final_noisy_overlap_list[idx,nidx]=get_overlap(true_data[:Npatts,:data_shape],decoded_data[:Npatts,:data_shape])


100%|██████████| 36/36 [32:06<00:00, 53.50s/it]


### Plotting/saving

In [ ]:
plt.plot(Npatts_list,np.average(final_noisy_overlap_list,axis=1), label='with sgn nonlin')
plt.plot(Npatts_list,np.average(final_clean_overlap_list,axis=1), label='without sgn nonlin')
# plt.errorbar(Npatts_list,np.average(final_noisy_overlap_list,axis=1),yerr=np.std(final_noisy_overlap_list,axis=1), label='noisy, final')
# plt.errorbar(Npatts_list,np.average(final_clean_overlap_list,axis=1),yerr=np.std(final_clean_overlap_list,axis=1), label='clean, final')
plt.xlabel('Number of patterns')
plt.ylabel('Overlap')
plt.legend()

In [ ]:
np.save("seq_wo_sgn_overlap_list_Np_"+str(Np)+"_Ns_"+str(Ns)+"_Ng_"+str(Ng)+"_nruns_"+str(nruns)+".npy", final_clean_overlap_list)
np.save("seq_w_sgn_overlap_list_Np_"+str(Np)+"_Ns_"+str(Ns)+"_Ng_"+str(Ng)+"_nruns_"+str(nruns)+".npy", final_noisy_overlap_list)
np.save("Npatts_list_seq.npy", Npatts_list)

